### Download and Preprocess data

In [1]:
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip

In [ ]:
!python preprocess_data.py -i data/OpportunityUCIDataset.zip -o oppChallenge_gestures.data

### Setup

In [17]:
import numpy as np
import _pickle as cp
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense,Reshape
from keras.layers import LSTM
from keras.layers import Flatten,Input
from keras.layers import Dropout,Permute,Lambda
from keras.layers.convolutional import Conv1D,Conv2D
from keras import optimizers
from keras.utils import np_utils
from keras.models import Model

from sliding_window import sliding_window

NB_SENSOR_CHANNELS = 113
NUM_CLASSES = 18
SLIDING_WINDOW_LENGTH = 24
FINAL_SEQUENCE_LENGTH = 8
SLIDING_WINDOW_STEP = 12
BATCH_SIZE = 128
NUM_FILTERS = 64
FILTER_SIZE = 5
NUM_UNITS_LSTM = 128

### Load the sensor data

In [2]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
X_train, y_train, X_test, y_test = load_dataset('data/oppChallenge_gestures.data')

Loading data...
 ..from file data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)


### Segmentation and Reshaping

In [3]:
assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x, (ws, data_x.shape[1]), (ss, 1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y, ws, ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)

# Data is reshaped
X_train = X_train.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)) # for input to Conv1D
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)) # for input to Conv1D
y_train = np_utils.to_categorical(y_train) # one-hot encoding
y_test = np_utils.to_categorical(y_test) # one-hot encoding

print(" ..after sliding and reshaping, train data: inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
print(" ..after sliding and reshaping, test data : inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

 ..after sliding and reshaping, train data: inputs (46495, 24, 113), targets (46495, 18)
 ..after sliding and reshaping, test data : inputs (9894, 24, 113), targets (9894, 18)


![](https://github.com/yminoh/DeepConvLSTM_Python3/raw/master/architecture_of_DeepConvLSTM.png)  
<div style="text-align: center">Figure 3 from Ordonez and Roggen, 2016</div>

### Build and train the Kears layers

In [18]:
rmp = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

inputs = Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS), name='input2')
print('shape of input ',inputs.shape)
outputs = Reshape((1,SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))(inputs)
outputs = Conv2D(NUM_FILTERS,kernel_size=[FILTER_SIZE, 1],
        strides=(1, 1), padding='VALID', activation=None, data_format = 'channels_first')(outputs)
print('shape after conv2d1 ',outputs.shape)
outputs = Conv2D(NUM_FILTERS,kernel_size=[FILTER_SIZE, 1],
        strides=(1, 1), padding='VALID', activation=None, data_format='channels_first')(outputs)
print('shape after conv2d2 ',outputs.shape)
outputs = Conv2D(NUM_FILTERS,kernel_size=[FILTER_SIZE, 1],
        strides=(1, 1), padding='VALID', activation=None, data_format='channels_first')(outputs)
outputs = Conv2D(NUM_FILTERS,kernel_size=[FILTER_SIZE, 1],
        strides=(1, 1), padding='VALID', activation=None, data_format='channels_first')(outputs)
print('shape before permute ',outputs.shape)

outputs = Permute((2,1,3))(outputs)
print('shape after permute ',outputs.shape)
outputs = Reshape((8,64*113))(outputs)
print('shape before LSTM ',outputs.shape)
outputs = LSTM(NUM_UNITS_LSTM,return_sequences=True)(outputs)
outputs = LSTM(NUM_UNITS_LSTM,return_sequences=True)(outputs)
outputs = Reshape((-1, NUM_UNITS_LSTM))(outputs)

def reduce_dim(x):
    return x[:,-1:,:]

outputs = Lambda(reduce_dim)(outputs)
outputs = Flatten()(outputs)
outputs = Dense(NUM_CLASSES, activation='softmax')(outputs)
# outputs = Reshape((-1, NUM_CLASSES))(outputs)

# def reduce_dim(x):
#     return x[:,-1:,1]

# outputs = Lambda(reduce_dim)(outputs)
model = Model(inputs=inputs, outputs=outputs)
#adam = keras.optimizers.Adam(lr=0.001, beta_1=0.5, beta_2=0.9, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=rmp, metrics=['accuracy'])

model.summary()

shape of input  (?, 24, 113)
shape after conv2d1  (?, 64, 20, 113)
shape after conv2d2  (?, 64, 16, 113)
shape before permute  (?, 64, 8, 113)
shape after permute  (?, 8, 64, 113)
shape before LSTM  (?, 8, 7232)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input2 (InputLayer)          (None, 24, 113)           0         
_________________________________________________________________
reshape_21 (Reshape)         (None, 1, 24, 113)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 64, 20, 113)       384       
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 64, 16, 113)       20544     
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 64, 12, 113)       20544     
______________________________________________________________

![](https://github.com/yminoh/DeepConvLSTM_Python3/raw/master/Table1.png)  
<div style="text-align: center">Table 1 from Ordonez and Roggen, 2016</div>

In [19]:
model.fit(X_train, y_train, validation_split=0.2, 
          epochs=10, batch_size=BATCH_SIZE, verbose=1)

W0909 09:41:16.038125  7460 deprecation.py:323] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 37196 samples, validate on 9299 samples
Epoch 1/10
37196/37196 [==============================] - 47s 1ms/step - loss: 1.5022 - acc: 0.6740 - val_loss: 3.6234 - val_acc: 0.0139
Epoch 2/10
37196/37196 [==============================] - 41s 1ms/step - loss: 1.5016 - acc: 0.6719 - val_loss: 1.1475 - val_acc: 0.7736
Epoch 3/10
37196/37196 [==============================] - 40s 1ms/step - loss: 1.4914 - acc: 0.6763 - val_loss: 1.1397 - val_acc: 0.7736
Epoch 4/10
37196/37196 [==============================] - 39s 1ms/step - loss: 1.4909 - acc: 0.6763 - val_loss: 1.1582 - val_acc: 0.7736
Epoch 5/10
37196/37196 [==============================] - 40s 1ms/step - loss: 1.4908 - acc: 0.6763 - val_loss: 1.1554 - val_acc: 0.7736
Epoch 6/10
37196/37196 [==============================] - 41s 1ms/step - loss: 1.4905 - acc: 0.6763 - val_loss: 1.1490 - val_acc: 0.7736
Epoch 7/10
37196/37196 [==============================] - 41s 1ms/step - loss: 1.4902 - acc: 0.6763 - val_loss: 1.1557 - val_acc:

### Evaluate the model

In [20]:
test_pred = np.argmax(model.predict(X_test), axis=1)
test_true = np.argmax(y_test, axis=1)
np.unique(test_pred)

array([0], dtype=int64)

In [21]:
import sklearn.metrics as metrics
print("\tTest fscore:\t{:.4f} ".format(metrics.f1_score(test_true, test_pred, average='weighted')))

	Test fscore:	0.7564 


C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
import time

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]
        
# Classification of the testing data
print("Processing {0} instances in mini-batches of {1}".format(X_test.shape[0],BATCH_SIZE))
test_pred = np.empty((0))
test_true = np.empty((0))
start_time = time.time()
for batch in iterate_minibatches(X_test, y_test, BATCH_SIZE):
    inputs, targets = batch
    y_pred, = test_fn(inputs)
    test_pred = np.append(test_pred, y_pred, axis=0)
    test_true = np.append(test_true, targets, axis=0)